In [ ]:
import pandas as pd
import os

In [ ]:
base_folder = "estimation"  # Or wherever your data is stored
sizes = [200, 500, 1000]

for model in parameters:
    all_data = []  # List to store DataFrames for different sizes

    for size in sizes:
        # Load existing DataFrame for the current size
        size_folder = os.path.join(base_folder, model, f"size_{size}")
        estimstat_filename = f"estimstat_{model}_{size}.csv"
        estimstat_filepath = os.path.join(size_folder, estimstat_filename)
        estimstat_df = pd.read_csv(estimstat_filepath)

        # Add a 'Size' column to the DataFrame
        estimstat_df['Size'] = size  

        # Append the DataFrame to the list
        all_data.append(estimstat_df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True) 

    # Save the combined DataFrame to a new CSV file
    combined_filename = f"combined_data_{model}.csv"
    combined_filepath = os.path.join(base_folder, model, combined_filename)
    combined_df.to_csv(combined_filepath, index=False)  

    print(f"Combined data for model {model} saved to: {combined_filepath}")